In [20]:
import cv2
import numpy as np
import time
import base64
import socketio
import winsound as sd

In [21]:
def BeepSound():
    fr = 2000    # range : 37 ~ 32767
    du = 2000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

In [22]:
"""
@는 데코레이터임
https://bluese05.tistory.com/30 설명
"""
sio = socketio.Client()

@sio.event
def wop(data):
    #받아온 데이터를 체크 처리
    print(data)
    #BeepSound()
    

In [23]:
#카메라 열기 
cap = cv2.VideoCapture(0)

#카메라 Width Height 설정
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

#인코딩용 전송 파라미터 설정
encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 90]

#소켓통신을 위한 준비
isSocket = True

#소켓 호출 에러시 처리하기 위한 부분
try:
    sio.connect('http://localhost:746')
except:
    isSocket = False

if isSocket == True:
    while True:
        time.sleep(0.1)
        #카메라로 부터 사진 한장 읽기 
        ret, frame = cap.read()

        frame = cv2.flip(frame, 1)
        
        scResult, scFrame = cv2.imencode('.jpg', frame, encode_param)        
        scData = base64.b64encode(scFrame)

        #소켓으로 이미지 전송
        sio.emit('streaming', scData)
        
        cv2.imshow('Person Detect Client', frame)

        if cv2.waitKey(1) == 13:
            break
            
        #소켓을 통해서 사람 수 받기
        sio.on('wop', wop)

    sio.disconnect()
    cap.release()
    cv2.destroyAllWindows()    
else:
    print("소켓접속 불량")